In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense


# Importar el data set
dataset = pd.read_csv('atp_matches_2022.csv')

In [2]:
X = dataset.iloc[:, [7, 15]].values
y = dataset.iloc[:, 7].values

In [3]:
# Reemplazar todos los valores NaN en 'X' por ceros
X = np.nan_to_num(X)

In [4]:
# Crear el codificador
encoder = OneHotEncoder(sparse_output=False)

# Ajustar el codificador con los datos de ambos jugadores
encoder.fit(X[:, [0, 1]].reshape(-1, 1))

# Transformar las columnas de los jugadores
player1_onehot = encoder.transform(X[:, 0].reshape(-1, 1))
player2_onehot = encoder.transform(X[:, 1].reshape(-1, 1))

# Concatenar las columnas one-hot con el resto de tus datos
X = np.concatenate([player1_onehot, player2_onehot, X[:, 2:]], axis=1)

In [5]:
# Crear una matriz de booleanos aleatorios del mismo tamaño que tus datos
swap = np.random.rand(len(X)) > 0.5

# Intercambiar las posiciones de los jugadores en 'X' donde 'swap' es True
X[swap, :408], X[swap, 408:816] = X[swap, 408:816], X[swap, :408].copy()

In [6]:
# Supongamos que 'winners' es una lista que contiene los códigos de los jugadores que ganaron cada partido
winners = y

# Transformar el vector objetivo 'y'
y = encoder.transform(winners.reshape(-1, 1))


In [7]:
from sklearn.model_selection import train_test_split

# Luego, dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
# Definimos el modelo
model = Sequential()

# Añadir la capa de entrada y la primera capa oculta
model.add(Dense(units=128, activation='relu', input_dim=X_train.shape[1]))

# Añadir la segunda capa oculta
# model.add(Dense(units=32, activation='tanh'))

# Añadir la capa de salida
model.add(Dense(units=y.shape[1], activation='softmax'))

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=150, batch_size=32)

Epoch 1/150
73/73 [==============================] - 1s 3ms/step - loss: 5.8028 - accuracy: 0.1084
Epoch 2/150
73/73 [==============================] - 0s 2ms/step - loss: 4.8533 - accuracy: 0.1595
Epoch 3/150
73/73 [==============================] - 0s 3ms/step - loss: 4.2695 - accuracy: 0.2970
Epoch 4/150
73/73 [==============================] - 0s 2ms/step - loss: 3.7911 - accuracy: 0.4385
Epoch 5/150
73/73 [==============================] - 0s 3ms/step - loss: 3.2508 - accuracy: 0.5607
Epoch 6/150
73/73 [==============================] - 0s 2ms/step - loss: 2.7017 - accuracy: 0.6412
Epoch 7/150
73/73 [==============================] - 0s 2ms/step - loss: 2.2060 - accuracy: 0.6725
Epoch 8/150
73/73 [==============================] - 0s 3ms/step - loss: 1.7999 - accuracy: 0.7038
Epoch 9/150
73/73 [==============================] - 0s 2ms/step - loss: 1.4762 - accuracy: 0.7381
Epoch 10/150
73/73 [==============================] - 0s 2ms/step - loss: 1.2258 - accuracy: 0.7630
Epoch 11/

In [9]:
# Hacemos la predicción
prediccion_test = model.predict(X_test)
# La predicción es un array con las probabilidades de cada clase
# Podemos obtener la clase con la probabilidad más alta con argmax
clase_predicha_test = np.argmax(prediccion_test, axis=1)

# Convertimos clase_predicha_test en un array 2D
clase_predicha_test_2D = np.eye(encoder.categories_[0].shape[0])[clase_predicha_test]

# Usamos el codificador para transformar la clase predicha a su etiqueta original
etiqueta_predicha_test = encoder.inverse_transform(clase_predicha_test_2D)

print(etiqueta_predicha_test)

19/19 [==============================] - 0s 1ms/step
[[106415]
 [126846]
 [105676]
 [132283]
 [105430]
 [126610]
 [207989]
 [104665]
 [126774]
 [106421]
 [134868]
 [105173]
 [134770]
 [126214]
 [122330]
 [106426]
 [128034]
 [105870]
 [100644]
 [106421]
 [126205]
 [106233]
 [105575]
 [105311]
 [104926]
 [126127]
 [200175]
 [106043]
 [126774]
 [105376]
 [106421]
 [106421]
 [132283]
 [200000]
 [122330]
 [105138]
 [126610]
 [106210]
 [200000]
 [126203]
 [106186]
 [207989]
 [200384]
 [200615]
 [133430]
 [105138]
 [207518]
 [105676]
 [111513]
 [106426]
 [104745]
 [126523]
 [200514]
 [111575]
 [200282]
 [105777]
 [105227]
 [208029]
 [200000]
 [206173]
 [200175]
 [104665]
 [105173]
 [111460]
 [125802]
 [126774]
 [207518]
 [126094]
 [104745]
 [106218]
 [106283]
 [111460]
 [105173]
 [100644]
 [104468]
 [104925]
 [208029]
 [126094]
 [111442]
 [126774]
 [202103]
 [126205]
 [144817]
 [200282]
 [200000]
 [126774]
 [200670]
 [133430]
 [104925]
 [128034]
 [133430]
 [134770]
 [105575]
 [207989]
 [10604

In [10]:
# Usamos el codificador para transformar las etiquetas codificadas a sus etiquetas originales
etiqueta_y_test = encoder.inverse_transform(y_test)

print(etiqueta_y_test)

[[106415]
 [104926]
 [105676]
 [132283]
 [106298]
 [200282]
 [208057]
 [200005]
 [126774]
 [105227]
 [134472]
 [105173]
 [134770]
 [126214]
 [104918]
 [200282]
 [128034]
 [209209]
 [100644]
 [104792]
 [206909]
 [202104]
 [105575]
 [105311]
 [104926]
 [126127]
 [200175]
 [134770]
 [126774]
 [105332]
 [200005]
 [106421]
 [132283]
 [200000]
 [122330]
 [105138]
 [126610]
 [106210]
 [208414]
 [126203]
 [106186]
 [207989]
 [106234]
 [134868]
 [133430]
 [105138]
 [200247]
 [105676]
 [111513]
 [126214]
 [104745]
 [106210]
 [200514]
 [111575]
 [200282]
 [105777]
 [200175]
 [208029]
 [200000]
 [105807]
 [200175]
 [104665]
 [105173]
 [105575]
 [125802]
 [126774]
 [125802]
 [126094]
 [104925]
 [106218]
 [111153]
 [105575]
 [105173]
 [100644]
 [104468]
 [104925]
 [106234]
 [206173]
 [123755]
 [126774]
 [126205]
 [126205]
 [111456]
 [200282]
 [200000]
 [202385]
 [200325]
 [133430]
 [104925]
 [128034]
 [133430]
 [134770]
 [105575]
 [207989]
 [126774]
 [122330]
 [134770]
 [200325]
 [104755]
 [126207]


In [11]:
# Veamos cuántos partidos ha acertado para el test
aciertos = 0
for i in range(len(etiqueta_predicha_test)):
    if etiqueta_predicha_test[i] == etiqueta_y_test[i]:
        aciertos += 1
print(f"Precisión del test: {np.round(aciertos / len(etiqueta_predicha_test)*100, 2)}%")

Precisión del test: 62.67%
